In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import ipywidgets as widgets
from IPython.display import display

<figure>
  <img src="the-simpsons.png" alt="the-simpsons.png"/>
  <figcaption>(c) 20th Century Fox Television</figcaption>
</figure>

# Chat with Simpsons

Start your conversation with Simpsons chatbot!

In [2]:
btn_launch = widgets.Button(description="Start")

In [3]:
def load_model(change):
    model = AutoModelForCausalLM.from_pretrained("arampacha/DialoGPT-medium-simpsons")
    tokenizer = AutoTokenizer.from_pretrained("arampacha/DialoGPT-medium-simpsons")
    
    for step in range(5):
        # encode the new user input, add the eos_token and return a tensor in Pytorch
        new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

        # append the new user input tokens to the chat history
        bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

        # generated a response while limiting the total chat history to 1000 tokens, 
        chat_history_ids = model.generate(bot_input_ids, 
                                          max_length=1000, 
                                          pad_token_id=tokenizer.eos_token_id,
                                          do_sample=True,
                                          top_p=0.9)

        # pretty print last ouput tokens from bot
        print("SimpsonsBot: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
        
btn_launch.on_click(load_model)

In [4]:
class FakeModel:
    def __call__(self, x):
        return x[::-1]

In [5]:
model = FakeModel()

In [6]:
btn_test = widgets.Button(description="Test")

In [7]:
output_text = widgets.Textarea(placeholder="Start dialog", disabled=True, layout={'height': '100%', 'width':'99%'})
user_input = widgets.Text(description="Input:", layout={'width':'99%'})

In [8]:
# output_text = widgets.HTML(
#     value="",
#     placeholder="Start the dialog by typing something below"
# )

In [9]:
def write_to_output(sender):
    output_text.value += f"User: {user_input.value}\n"
    output_text.value += f"Bot:  {model(user_input.value)}\n"
    user_input.value=""
    
user_input.on_submit(write_to_output)

In [10]:
display(widgets.VBox([output_text, user_input], layout={'height': '500px', 'width':'500px'}))

In [ ]:
# for step in range(5):
#     # encode the new user input, add the eos_token and return a tensor in Pytorch
#     new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

#     # append the new user input tokens to the chat history
#     bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

#     # generated a response while limiting the total chat history to 1000 tokens, 
#     chat_history_ids = model.generate(bot_input_ids, 
#                                       max_length=1000, 
#                                       pad_token_id=tokenizer.eos_token_id,
#                                       do_sample=True,
#                                       top_p=0.9)

#     # pretty print last ouput tokens from bot
#     print("SimpsonsBot: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
